# Divide & conquer

Divide & conquer is a method for designing algorithms that solve problems by breaking the problem into smaller subproblems and then solving those smaller problems by dividing further until some base case is reached when the problem is small enough. Once the base case is reached we solve that subproblem and then use that solution to solve the larger problem by combining it with solutions to the other smaller subproblems we have solved. Merge sort is a nice example of this that we will go over. As you can probably guess from this description divide & conquer is very closely tied to the concept of recursion.


## Binary search

Perhaps the simplest example of a divide and conquer algorithm is binary search. The algorithm solves the following problem: Given a *sorted* (ascending order) list of numbers $A$ of size $n$ how quickly can we determine of an arbitrary target value $t$ is in the list and find its position in the list? The algorithm is given below.

```{prf:algorithm} Binary Search
:class: dropdown
:label: bin-search
**procedure** $\text{binary_search}(A,t)$:

**Inputs** Given a list of $A[1,...,n]$ and a target $t$

**Output** The position of $t$ in the list if it exists in the list

1. $L = 1$
2. $R = n$
3. while $L\leq R$:
   1. $m = \lfloor\frac{L+R}{2}\rfloor$
   2. if $A[m]<t$:
      1. $L=m+1$
   3. else if $A[m]>t$:
      1. $R=m-1$
   4. else if $A[m]=t$:
      1. return $m$
4. return unsuccessful 
```

At each iteration we are checking the value in the value in the middle of the list to see if it is greater than, equal to or less than $t$. If the value is equal to $t$ then we are done. If the value is greater than $t$ then we update the iteration to only look at the elements in the list that come before the middle value. Why? Because the list is sorted in ascending order if the middle element is greater than $t$ then we *know* all values that come after the middle element will also be greater than $t$ since they must be equal to or greater than the middle element. The same logic applied in the case where the middle element is less than $t$ except in that case in the next iteration we look at only the elements that come *after* the middle element. 

What is the time complexity of this algorithm? Well during each iteration we just compare two numbers which takes $O(1)$ time. But how many iterations, or recursion calls if we implement this with recursion, will we have to compute before we finish? Well the algorithm terminates when we have only 1 item left in the list and since we remove half the elements in the list at each iteration we need to simply answer: how many times do we need to divide $n$ by $2$ before we reach $1$? The simple algebriac equation that descirbes this is given by

$$
\frac{n}{2^x} = 1
$$

solving this for $x$ yields $x=\log_2{n}$ so in the worse case where the element is not in the list we must complete $\log_2{n}$ iterations. So the time complexity of binary search is $O(\log_2{n})$ which is incredibly efficient. 

Alternatively if we had implemented this algorithm using recursion then the time complexity for solving a problem of size $n$ would be described by the recurrence relation

$$
T(n) = T\left(\frac{n}{2}\right) + O(1)
$$

which can be read as saying: to solve a problem of size $n$ requires we perform a constant number of operations, which is captured by the $O(1)$, as well as solve a *single* problem of half the size which is captured by the $T\left(\frac{n}{2}\right)$.

## Integer multiplication  


The first problem divide & conquer methods are applied to in {cite:p}`dasgupta2008algorithms` is integer multiplication. 

To multiple two complex numbers $g=a+ib$ and $h=c+id$ we compute

$$
(a+ib)(c+id)= ac-bd + (bc+ad)i
$$

which involves 4 multiplications of real numbers. Carl Gauss, the famous mathematician, discovered that we can actually reduce this to 3 since 

$$
bc+ad = (a+b)(c+d) - ac - bd
$$

our formula now becomes

$$
\begin{align*}
(a+ib)(c+id) &= ac-bd + (bc+ad)i \\
&= ac-bd + ((a+b)(c+d) - ac - bd)i.
\end{align*}
$$

It seems like this involves more multiplication but actually we now just need to compute $ac$, $bd$ and $(a+b)(c+d)$ since $ac$ and $bd$ appear twice in the expression so we have 3 *unique* multiplications of real numbers.


This might seem like minimal improvement but lets see what happens when we apply recursion and switch to integer multiplication. Suppose $y$ and $x$ are $n$-bit integers where $n$ is a power of 2. First lets start by splitting $x$ and $y$ into two halves that are $\frac{n}{2}$ bits each such that 

$$
x = 2^{\frac{n}{2}} x_L + x_R \quad \text{ and } \quad y = 2^{\frac{n}{2}} y_L + y_R.
$$

For example if  $x=10110110_2$ then $x_L=1011_2$ and $x_R=0110_2$ and $x=(2^{\frac{n}{2}} \times 1011_2) + 0110_2$, (note the subscript of 2 means the number is written in base 2 i.e. binary).

```{note}
:class: dropdown
Note that we multiply the left most bits by $2^{n/2}$ since these are the $n/2$ most significant bits since we're using big endian convention. 
As an explicit example for the $8$-bit number 

$$
\begin{align*}
x &= 182_{10}\\
&=10110110_2 \\ 
&= (1\times2^7)+(0\times2^6)+(1\times2^5)+(1\times2^4)+(0\times2^3)+(1\times2^2)+(1\times2^1)+(0\times2^0)
\end{align*}
$$

since the leftmost bits are the most significant we must have $x_L=2^{4}\times1011_2 =2^{\frac{8}{2}}\left[(1\times2^3)+(0\times2^2)+(1\times2^1)+(1\times2^0)\right]$ if we are to have $x_R=0110_2$ and satisfy the equation $x_L+x_R=x$.
```

Now the product of $x$ and $y$ is given by

$$
\begin{align*}
xy &= (2^{\frac{n}{2}} x_L + x_R)(2^{\frac{n}{2}} y_L + y_R)  \\
   &= 2^nx_Ly_L + 2^{\frac{n}{2}}(x_Ly_R + x_Ry_L) + x_Ry_R
\end{align*}
$$

The addition will take linear time, in the number of bits, and so to will the power of 2 multiplications since it is just a bit shift to the left i.e. `a << n` or `a << n_2` in  `Python`. The important operation are the 4 multiplications of the $\frac{n}{2}$-bit numbers. Notice that we have *divided* the problem into 4 subproblems each of which are *half* the size i.e. $\frac{n}{2}$. We can now recursively perform the same routine for the 4 new multiplications which would further divide the problem into subproblem of smaller size. 
We can describe the runtime (time complexity) of these recursive algorithms using *recurrence relations* which we cover in the next section. The recurrence relation for this algorithm is given by

$$
T(n) = 4T\left(\frac{n}{2}\right) + O(n)
$$

we multiply by 4 since we are creating 4 new subproblems and multiple by $T\left(\frac{n}{2}\right)$ since the new subproblems have size $\frac{n}{2}$. The additional $O(n)$ is there to capture the linear time complexity of additions and leftward bit shifts. Solving this recurrence relation for $T(n)$ we get the solution is $T(n)=O(n^2)$. This is the same complexity as the grade-school multiplication method so we have no real improvement with this new recursive algorithm.

We can improve this algorithm by using the insight from Gauss that we laid out earlier. Since $x_Ly_R + x_Ry_L=(x_L+x_R)(y_L+y_R)-x_Ly_L-x_Ry_R$ we can reduce the 4 multiplications needed down to 3 resulting in a more efficient algorithm. 

The pseudocode and for the algorithm are given below.

```{prf:algorithm} Integer Multiplication
:class: dropdown
:label: gauss-int-mult 
**procedure** $\text{gauss_int_mult}(x,y,n)$:

**Inputs** Given $n$-bit integers $x$ and $y$

**Output** Their integer product $xy$

1. if $n=1$: 
   1. return $xy$
2. $x_L = \text{ leftmost } \lceil \frac{n}{2} \rceil \text{ bits of } x$
3. $x_R = \text{ rightmost } \lfloor \frac{n}{2} \rfloor \text{ bits of } x$
4. $y_L = \text{ leftmost } \lceil \frac{n}{2} \rceil \text{ bits of } y$
5. $y_R = \text{ rightmost } \lfloor \frac{n}{2} \rfloor \text{ bits of } y$
6. $P_1 = \text{fast_int_mult}(x_L,y_L)$ $\quad \triangleright$ Compute $x_Ly_L$
7. $P_2 = \text{fast_int_mult}(x_R,y_R)$ $\quad \triangleright$ Compute $x_Ry_R$
8. $P_3 = \text{fast_int_mult}(x_L+x_R,y_L+y_R)$ $\quad \triangleright$ Compute $(x_L+x_R)(y_L+y_R)$
9. return $2^{n} P_1 + 2^{n}(P_3 - P_1 - P_2) + P_2$

```

The time complexity for this algorithm is given by the recurrence relation

$$
T(n) = 3T\left(\frac{n}{2}\right) + O(n)
$$

which has the solution $T(n) = O(n^{1.59})$ which is a very nice improvement.

## Solving recurrence relations

A recurrence relation, which we'll call recurrence for short from now on, is an equation that describes a function in terms of it's values on other, usually smaller, arguments. To solve these recurrences there are three main methods described below {cite:p}`cormen2022introduction`.

### Substitution

Given a recursion based algorithm we guess the solution and use induction to prove it is correct. For example for the recurrence relation

$$
T(n) = T\left(\frac{4n}{5}\right) + O(n)
$$


### Recursion-tree 

Given a recursion based algorithm you draw out a the recursion as a tree with with the number of branches at each node being the number of subproblems being created, the depth being the levels of recursion and the nodes of the tree representing the cost of an operation at that level of recursion. By examining the tree you then attempt to write out the expression for the time complexity by summing up the nodes.

### Master method

Given a recursion based algorithm if we can express it's runtime as a recurrence of the form

$$
T(n) = aT\left(\left\lceil\frac{n}{b}\right\rceil\right) + O(n^d)
$$

where $n$ is the size of the initial problem, $a$ is the number of subproblems that we divide the problem into at each recursion level, $\frac{n}{b}$ is the size of the subproblems and $O(n^d)$ is the time needed to combine the solutions of said subproblems into solutions for the larger problems we can use the *master theorem* to solve the recurrence. 

```{prf:theorem} Master Theorem
:class: dropdown
Given the recurrence 

$$
T(n) = aT\left(\left\lceil\frac{n}{b}\right\rceil\right) + O(n^d)
$$

the solution $T(n)$ is given by

$$
T(n) = \begin{cases}
  O(n^d)  & \text{if } d > \log_b(a)\\
  O(n^d\log(n))  & \text{if } d = \log_b(a)\\
  O(n^{\log_b(a)})  & \text{if } d < \log_b(a)\\
\end{cases}
$$
which is very useful. The proof along with the theorem is given in {cite:p}`dasgupta2008algorithms`.
```

## Square matrix multiplication

### Naive matrix multiplication  

Let's go over an example from {cite:p}`cormen2022introduction` now. Let $A$ and $B$ be $n\times n$ matrices. Their product $C=AB$ is given by (recall that this is equivalent to taking the dot product between the rows of $A$ with the columns of $B$)

$$
C_{ij} = \sum_{k=1}^{n} A_{ik}B_{kj}
$$

where $C_{ij}$ is the element of matrix $C$ at row $i$ column $j$. The pseudocode for an algorithm that computes this formula is given below.

From the triply nested for-loops we can tell the runtime of this algorithm will be $\Theta(n^3)$, since for $n$ iterations we are performing $n$ operations $n$ times. 

We can however apply the divide and conquer method to this problem by utilizing *matrix partitioning*. Recall that for two $n\times n$ matrices $A$ and $B$, if $n$ is even, we can partition the matrices as 

$$
A = \begin{bmatrix}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{bmatrix} 
\text{ and } 
B = \begin{bmatrix}
B_{11} & B_{12}\\
B_{21} & B_{22}
\end{bmatrix} 
$$

where the blocks $A_{ij}$ and $B_{ij}$ are simply four $\frac{n}{2} \times \frac{n}{2}$ square block submatrices. We can now express the matrix product in terms of the standard matrices product formula except this time we treat the block matrices as scalars in a sense

$$
\begin{align*}
AB &= \begin{bmatrix}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{bmatrix}
\begin{bmatrix}
B_{11} & B_{12}\\
B_{21} & B_{22} 
\end{bmatrix} \\
&= \begin{bmatrix}
A_{11}B_{11} + A_{12}B_{21} &  A_{11}B_{12} + A_{12}B_{22} \\
A_{21}B_{11} + A_{22}B_{21} & A_{21}B_{12} + A_{22}B_{22} 
\end{bmatrix}.
\end{align*}
$$

Now we can begin to see how divide and conquer can be applied. Notice that we now have 4 smaller problems now i.e. computing the 4 expressions 

$$
C_{11} = A_{11}B_{11} + A_{12}B_{21}\\
C_{12} = A_{11}B_{12} + A_{12}B_{22}\\
C_{21} = A_{21}B_{11} + A_{22}B_{21}\\
C_{22} = A_{21}B_{12} + A_{22}B_{22} 
$$

which involve 4 addition and 8 multiplication of *smaller* matrices now i.e. matrices of size $\frac{n}{2}\times \frac{n}{2}$. The divide and conquer algorithm is given below along with a python implementation of it.

```{prf:algorithm} Naive Matrix Multiplication
:class: dropdown
:label: naive-mat-mult 
**procedure** $\text{naive_mat_mult}(A,B)$:

**Inputs** Given two $n\times n$ matrices $A$ and $B$

**Output** Their matrix product $C=AB$

1. Initialize $C$ to an all zeros matrix of size $n\times n$
2. for $i=0$ to $n$:
   1. for $j=0$ to $n$:
      1. for $k=0$ to $n$:
         1. $C[i][j] = C[i][j] + A[i][k] \times B[k][j]$
3. return $C$

```

```{prf:algorithm} D&C Matrix Multiplication
:class: dropdown
:label: d&c-mat-mult 
**procedure** $\text{dc_mat_mult}(A,B,C, n)$:

**Inputs** Given two $n\times n$ matrices $A$ and $B$ and an all zeros matrix of size $n\times n$

**Output** Their matrix product $C=AB$

1. if $n=1$:
   1. $C[0][0] = C[0][0] + A[0][0]\times B[0][0]$ $\quad \triangleright$ These are simply scalars since $n=1$
2. Partition $A$, $B$ and $C$ into the 12 $\frac{n}{2}\times \frac{n}{2}$ matrices $A_{11},A_{21},A_{21},A_{22},B_{11},B_{21},B_{21},B_{22},C_{11},C_{12},C_{21}$ and $C_{22}$ $\quad \triangleright$ Divide step
3. $\text{dc_mat_mult}(A_{11},B_{11},C_{11}, \frac{n}{2})$ $\quad \triangleright$ This computes $A_{11}B_{11}$ from $C_{11} = A_{11}B_{11} + A_{12}B_{21}$
4. $\text{dc_mat_mult}(A_{12},B_{21},C_{11}, \frac{n}{2})$ $\quad \triangleright$ This computes $A_{12}B_{21}$ from $C_{11} = A_{11}B_{11} + A_{12}B_{21}$
5. $\text{dc_mat_mult}(A_{11},B_{12},C_{12}, \frac{n}{2})$
6. $\text{dc_mat_mult}(A_{12},B_{22},C_{12}, \frac{n}{2})$
7. $\text{dc_mat_mult}(A_{21},B_{11},C_{21}, \frac{n}{2})$
8. $\text{dc_mat_mult}(A_{22},B_{21},C_{21}, \frac{n}{2})$
9. $\text{dc_mat_mult}(A_{21},B_{12},C_{22}, \frac{n}{2})$
10. $\text{dc_mat_mult}(A_{22},B_{22},C_{22}, \frac{n}{2})$
```

In [44]:
import numpy as np
np.random.seed(12)

def dc_mat_mult(A, B, C, n):
    if n == 1:
       C[0, 0] += (A[0,0] * B[0,0])
       return C

    n_2 = n // 2
    A11 = A[0:n_2, 0:n_2]
    A12 = A[0:n_2, n_2:n]
    A21 = A[n_2:n, 0:n_2]
    A22 = A[n_2:n, n_2:n]

    B11 = B[0:n_2, 0:n_2]
    B12 = B[0:n_2, n_2:n]
    B21 = B[n_2:n, 0:n_2]
    B22 = B[n_2:n, n_2:n]

    dc_mat_mult(A11, B11, C[0:n_2, 0:n_2], n_2)
    dc_mat_mult(A12, B21, C[0:n_2, 0:n_2], n_2)
    dc_mat_mult(A11, B12, C[0:n_2, n_2:n], n_2)
    dc_mat_mult(A12, B22, C[0:n_2, n_2:n], n_2)
    dc_mat_mult(A21, B11, C[n_2:n, 0:n_2], n_2)
    dc_mat_mult(A22, B21, C[n_2:n, 0:n_2], n_2)
    dc_mat_mult(A21, B12, C[n_2:n, n_2:n], n_2)
    dc_mat_mult(A22, B22, C[n_2:n, n_2:n], n_2)
    
    return C

n = 2**6 # size of matrix
A = np.random.rand(n, n)
B = np.random.rand(n, n)
C = np.zeros((n, n))

AB = A@B # compute product with numpy
C = dc_mat_mult(A, B, C, n) # compute product with our algo

print("Are the matrices the same?", "Yes" if np.allclose(AB, C) else "No")

Are the matrices the same? Yes


Now since we divide the problem into 8 subproblems at each recursion (the 8 matrix multiplications of the submatrices) of size $\frac{n}{2}$ and the operation of partitioning the matrices can be made to have complexity $O(1)$ if we just keep track of the indices of the submatrices we are working with, instead of creating a whole new set of matrices in memory each time we partition a matrix into blocks which has complexity $O(n^2)$, we can characterize the runtime of this algorithm with the recurrence

$$
T(n) = 8T\left(\frac{n}{2}\right) + O(1).
$$

Using the master theorem we have the solution of this recurrence is $O(n^3)$ so not an asymptotic improvement over the naive method but a useful example of how to use divide and conquer to solve this problem. There is, however, a divide and conquer algorithm for matrix multiplication, devised by Volker Strassen, that is more clever and has better a runtime.


### Strassen's algorithm

Strassen's algorithm achieves a speed up over both the prior methods by utilizing a clever trick akin to the one found by Gauss for integer multiplication. Specifically the algorithm reduces the number of matrix multiplications that need to be computed at each recursion level from 8 to 7! This is done by computing the product as {cite:p}`dasgupta2008algorithms`

$$
AB = \begin{bmatrix}
P_5 + P_4 - P_2 + P_6 & P_1 + P_2\\
P_3 + P_4 & P_1 + P_5 - P_3 - P_7
\end{bmatrix}
$$

where 

$$
\begin{align*}
&P_1=A_{11}(B_{12} - B_{22}), \qquad P_2=(A_{11} + A_{12})B_{22}, \\
&P_3=(A_{21}+A_{22})B_{11}, \qquad P_4=A_{22}(B_{21}+B_{22}), \\
&P_5=(A_{11}+A_{22})(B_{11}+B_{22}), \qquad P_6=(A_{12}-A_{22})(B_{21}+B_{22})\\
&P_7=(A_{11}-A_{21})(B_{11}+B_{12}).
\end{align*}
$$

The recurrence for this algorithms time complexity is now given by

$$
T(n) = 7T\left(\frac{n}{2}\right) + O(1).
$$

and from the master theorem we get the solution to this recurrence is $T(n)=O(n^{log_{2}(7)}) \approx O(n^{2.807})$. This may not seem like a huge improvement but as the matrix sizes get large e.g. in the millions, which is this very common for computations in fields such as deep learning, this algorithms runtime is about an order of magnitude faster. This is, to my knowledge as of writing in 2024, the fastest *practical* matrix multiplication algorithm. There are other algorithms that are asymptotically faster such as the Coppersmith–Winograd algorithm with a time complexity of $O(n^{2.373})$ but these are not nearly as practical as they are what are known as [galactic algorithms](https://en.wikipedia.org/wiki/Galactic_algorithm).

## Merge sort

Merge sort is one of the best divide and conquer algorithms for sorting a list of numbers. The idea behind merge sort is you have a list $L$ of numbers and you go about sorting it by first dividing the list into two halves of equal length. We then recursively sort each half of the list and then merge the two sorted lists. The merge sort algorithm is given below.

```{prf:algorithm} Merge sort
:class: dropdown
:label: merge-sort
**procedure** $\text{merge_sort}(L)$:

**Inputs** Given a list $L[1,...,n]$ of length $n$

**Output** The sorted list $L$ in ascending order

1. if $n>1$: $\quad \triangleright$ Divide $L$ into two smaller lists
   1. $L_1  = L\left[1,...,\lfloor\frac{n}{2}\rfloor\right]$ $\quad \triangleright$ Get first $\lfloor\frac{n}{2}\rfloor$ elements 
   2. $L_2  = L\left[\lfloor\frac{n}{2}\rfloor+1,...,n\right]$ $\quad \triangleright$ Get last $\lfloor\frac{n}{2}\rfloor$ elements 
   3. return $\text{merge}( \text{merge_sort}(L_1),  \text{ merge_sort}(L_2) )$
2. else: $\quad \triangleright$ When list has only 1 element return the element
   1. return $L[1]$


**procedure** $\text{merge}(L_1, L_2)$:

**Inputs** Given 2 lists $L_1[1,...,n]$ and $L_2[1,...,m]$ of length $n$ and $m$ respectively sorted in ascending order.

**Output** The  combined list $L$ sorted in ascending order

1. if $n=0$:
   1. return $L_2[1,...,m]$
2. if $m=0$:
   1. return $L_1[1,...,n]$
3. if $L_1[1] \leq L_2[1]$:
   1. return $L_1[1] \oplus \text{merge}(L_1[2,...,n], L_2[1,...,m])$
4. else:
   1. return $L_2[1] \oplus \text{merge}(L_2[1,...,n], L_2[2,...,m])$

Note $L_1[1] \oplus \text{merge}(L_1[2,...,n], L_2[1,...,m])$ means we concatenate the list returned from the $\text{merge}$ call to the end of the one element list $L_1[1]$.

```

Now the $\text{merge}$ function is the crucial component here as it defines how we combine the resulting sorted lists. Let's think about how we would combine two sorted (ascending order) lists $L_1$ and $L_2$ into a list $L$. Well since both lists are sorted when now that the first element of the combined list will either be the first element of $L_1$ or the first element of $L_2$. But why are we assuming the lists are sorted? It isn't super clear where the lists are getting sorted in the first place. So the key thing to think about is the case where $\text{merge_sort}$'s base case is reached. When this happens we just return a a list with one element. Once this happens we start to recurse back up the recursion tree. So the $\text{merge}$ function is called with the input lists being single element lists. At that point we just compare the elements in both lists and concatenate them based on which is smaller as is done by the last two conditional statements in the $\text{merge}$ function. After this we move up one level in the recursion tree $\text{merge}$ is called again but this time the lists have length 2. So we can see that once we get to the top of the recursion tree we will have two list whose sizes, $n$ and $m$ respectively, add up to the size adds up to the length of the original input list. Clearly the $\text{merge}$ function compares the first element of $L_1$ with the first element of $L_2$ and then compares the second element of $L_1$ with the second element of $L_2$ and so on. This means $\text{merge}$ does $O(n+m)$ operations meaning the time for the merge step is linear in the size of the two lists. So we take a problem of size $n$ and divide it into two subproblems of size $\frac{n}{2}$ and solve each of these subproblems in time $O(n)$ thus the runtime of merge sort is characterized by the recurrence

$$
T(n) = 2T\left(\frac{n}{2}\right) + O(n)
$$

from the master theorem we have that the solution to this recurrence is $T(n)=O(n\log(n))$ which means we can sort a list in time that is log-linear in the size of the list. Not bad and in fact this is, provably, the best we can do for a *comparison based* sorting algorithm {cite:p}`dasgupta2008algorithms`. See radix sort for a non-comparison based sorting algorithm with complexity $O(n)$ {cite:p}`cormen2022introduction`.

## Medians

We will now look at how to find the median of a list of *unsorted* (unsorted because if the list was sorted the problem would be trivial) numbers using the divide & conquer method. The median of a set of numbers is of course the number that is smaller than 50% of the numbers in the list and greater than or equal to the other 50% of the numbers in the list. To devise an algorithm for this problem it is actually useful to generalize it. The median, for a list of size $n$, can also be thought of as the $\lfloor\frac{n}{2}\rfloor$-th smallest number in the list. So perhaps we should generalize this problem into finding the $k$-th smallest number in a list of numbers and we'll call this problem *selection*.

### Randomized divide & conquer

So how can we devise an algorithm to find the $k$-th smallest number in an unsorted list $L$? Suppose we take a number $w$ and split $L$ into 3 lists: $L_l$ which is the list of elements in $L$ that are less than $w$, $L_w$ the list of elements in $L$ that are equal to $w$ and $L_g$ which is the list of elements in $L$ that are greater than $w$. As an example lets take the following list

$$
L = [2, 36, 5, 21, 8, 13, 11, 20, 5, 4, 1]
$$

and take $w=5$. We would then split the list into

$$
\begin{align*}
L_l &= [2, 4, 1] \\ L_w &= [5,5] \\ L_g &= [36, 21, 8, 13, 11, 20].
\end{align*}
$$

Now suppose we want the 8th smallest element of $L$. We know that this would be the 3rd smallest element of $L_g$. Why? Think of it like this: Where is the smallest element? Well it's certainly in $L_l$ since that is the list with all elements smaller than $w$. Where is the second smallest element? Also in $L_l$. The third smallest element is also in $L_l$. Now what about the fourth smallest? Well that would be in $L_w$ since the first 3 smallest elements are in $L_l$. What about the sixth smallest element? Well it can't be in $L_l$ since that contains the first 3 smallest, and it can't be in $L_w$ since that contains the next 2 smallest so it must be in $L_g$, and more importantly it will be the smallest number in $L_g$. The seventh smallest number can be found in the same way except this time it will be the *second* smallest element in $L_g$. So to put it in general terms if we want the $k$-th smallest element if $k \leq |L_l|$ then we know the $k$-th smallest element is in $L_l$ and we can simply ignore the other two list. If $k > |L_l|+|L_w|$ then we know the $k$-th smallest element is in $L_g$. And if neither condition is true then that means $|L_s|<k\leq |L_l|+|L_w|$ which means the element we want is in $L_w$ i.e. the element is $w$ itself. The pseudocode is given below.

```{prf:algorithm} Selection - $O(n)$ Expected Case Time
:class: dropdown
:label: selection (Linear Expected Case Time)
**procedure** $\text{selection}(L, k)$:

**Inputs** Given a list $L[1,...,n]$ of length $n$ and an integer $1\leq k \leq n$

**Output** The $k$-th smallest element in the list

1. Select pivot element $w$ by randomly sampling uniformly from $L$

2. Divide $L$ into the lists $L_l$, $L_w$ and $L_g$
3. if $k \leq |L_l|$:
   1. return $\text{selection}(L_l, k)$
4. if $k > |L_l| + |L_w|$:
   1. return $\text{selection}(L_g, k-|L_l| - |L_w|)$
5. else:
    1. return $w$

Now what about the time complexity of this algorithm? Well if we assume that the size of $L_l$ and $L_g$ are roughly equal to $\frac{1}{2}|L|$ at each divide step then we are dividing the problem into a subproblem of half the size *but* we must note that the process of splitting the list into the three sublists takes linear time in the number of elements since we must compare each of the $n$ elements with $w$ to decide which list it should go into. So the recurrence for this algorithm is given by 

$$
T(n) = T\left(\frac{n}{2}\right) + O(n)
$$

which from the master theorem has the solution $O(n)$! This is a great solution but it relies on a rather strong assumption: that we choose a value of $w$ that creates list $L_l$ and $L_g$ which have size roughly equal to $\frac{1}{2}$ the size of the original list. But we don't really have a way of selecting $w$ that guarantees this to be the case every time. And in fact if we did have such a method of picking $w$ then we could just use that method to find the median since the value of $w$ that would divide the list into lists $L_l$ and $L_g$ of $\frac{n}{2}$ size is the median! What if we just pick $w$ randomly (hence why this is a randomized divide & conquer algorithm)? The worst case scenario here is that we want to run the algorithm for $k=1$, so we want the smallest element in the list, and pick a $w$ that is the largest element in the list each time which would only reduce the size of the list by one at each recursive call. This means we would end up doing $n$ recursive calls and $n-i$ comparisons at each call where $i$ is the recursion level we are on. We would thus do

$$
\sum^{n}_{i=1}(n-i) 
$$

total comparisons which by induction can be shown to be equal to $\frac{n(n-1)}{2}$ for $n \in \mathbb{Z^+}$ so our worst case time complexity is $O(n^2)$ which is the time a naive approach of just comparing each element with every other one would take to find the median. However the chance that we pick badly like this at every step is incredibly low. For example the chance that we pick the largest element in a list $n$ elements, assuming each element is equally likely to be chosen, is $\frac{1}{n}$. The chance that we do it again once that element is removed is $\frac{1}{n-1}$ and so on until we get to a list of only one element. That means the probability of this worst case scenario occurring is


$$
\prod_{i=0}^{n-1}\frac{1}{n-i} = \frac{1}{n!}
$$

which very quickly approaches 0 as $n$ increases. 

But this is just the absolute worse case; it would be more useful to know the time complexity for the case where we pick a pivot that is somewhat good. For example what if we pick a $w$ which creates lists of size $\frac{3n}{4}$ that we then recursively run the algorithm on? Well in that case the recurrence is given by

$$
T(n) = T\left(\frac{3n}{4}\right) + O(n)
$$

and if we use the master theorem we will come to the surprising conclusion that the solution to this recurrence is also $O(n)$ and in fact is $O(n)$ for any recurrence relation of the form

$$
T(n) = T\left(bn\right) + O(n)
$$

where $b<1$! So this means we have some leeway in picking $w$. If we can find a way to pick a $w$ that is less than the $\frac{3n}{4}$-th largest element and greater than the $\frac{n}{4}$-th largest we will have an $O(n)$ algorithm for finding the median of an unsorted list.

How many elements are there in the list that are less than the $\frac{3n}{4}$-th largest element and greater than the $\frac{n}{4}$-th largest element (note that it doesn't matter if the list is sorted)? Well thats $\frac{3n}{4}-\frac{n}{4}=\frac{n}{2}$ elements. So if we pick a pivot randomly that means we have a $\frac{1}{2}$ chance of picking a pivot that is one of these elements. This means that on average we have to randomly select an element twice to get a good pivot. This is akin to flipping a fair coin where {cite:p}`dasgupta2008algorithms` show that on average we must flip the coin two times before we see a heads. Note that once we randomly select a candiadate for the pivot we must check to see it satisfies the condition of being less than the $\frac{3n}{4}$-th largest element and greater than the $\frac{n}{4}$-th largest element in the list which will take $O(n)$ time. So on average we will need to do this twice before we have a good pivot so on average we can expect a time complexity of $O(n)$ when choosing the pivot in this way and thus our recurrence form earlier will describe the runtime of our algorithm. 


### Tighter bound 

But this is just the *expected/average* runtime of the algorithm. Can we modify the algorithm to get an even tighter bound that shows that in the *worst* case we have a complexity of $O(n)$? Well it turns out we can by modifying how we select the pivot which is explained by Dr. Vigoda in lecture and in chapter 9 of {cite:p}`cormen2022introduction`. 

So recall that for our recurrence which is of the form

$$
T(n) = T\left(bn\right) + O(n)
$$

we just need $b$ to be less than 1 for this recurrence to solve to $O(n)$. So lets use this leeway to devise a way to find a good pivot that keeps us within this $b<1$ range. To this end we will devise an algorithm whose time complexity is described by the following recurrence

$$
\begin{align*}
T(n) &= T\left(\frac{3n}{4}\right) + \left(T\left(\frac{n}{5}\right) + O(n)\right) + O(n) \\
     &= T\left(\frac{3n}{4}\right) + T\left(\frac{n}{5}\right) + O(n) \quad \text{Recall $O(n)+O(n)=O(n)$}\\
     &= T\left(\left(\frac{3}{4}+\frac{1}{5}\right)n\right) +  O(n) \quad \text{Recall $f(ax)+f(bx)=f((a+b)x)$ for real-valued functions}
\end{align*}
$$

The extra terms $\left(T\left(\frac{n}{5}\right) + O(n)\right)$ will account for the time taken to find a good pivot at each recursion level. And note that this still solves to $O(n)$ since $\left(\frac{3}{4}+\frac{1}{5}\right)<1$. But where does this extra term come from? Well the strategy we will use to find the pivot is we will create a list $S$ of size $\frac{n}{5}$ and recursively call our $\text{selection}$ algorithm *on it* to find it's median which we will then us as out pivot.

#### Median of medians

The key insight is that we want the elements of $S$ to possess a certain property. That is we want each $x\in S$ to have the property that there are a few elements of $L$ which are less than or equal to it and a few elements of $L$ which are greater than or equal to it. In our case we will pick "a few" to mean $2$. But how will we ensure this?

So first we will take $L$ and divide it into $\frac{n}{5}$ smaller lists $G_1,G_2,...,G_{n/5}$ which each have $5$ elements in them. We will then *sort* each list. You might think that this ruins our time complexity of $O(n)$ since (comparison-based) sorting takes $O(n\log{n})$ time but you would be incorrect. This is because each list has a constant number of elements in it regardless of the input size $n$ i.e. $5$ elements. So it in fact takes $O(1)$ time to sort each list! Once each $G_i$ is sorted we find its median, $m_i$, which takes $O(1)$ time since the list is sorted (just look at the middle element) and create $S=[m_1,m_2,...m_{n/5}]$ from these medians. We will then select our pivot from this list by finding it's median by *recursively calling $\text{selection}$ on it* with $k=\frac{n}{10}$. $k=\frac{n}{10}$ since $S$ is of size $\frac{n}{2}$ the median is the $\frac{\frac{n}{5}}{2} = \frac{n}{10}$ smallest element. This whole process is described by the recurrence 

$$
T(n) = T\left(\frac{n}{5}\right) +  O(n)
$$

which were the extra terms we added to the recurrence of the expected time $\text{selection}$ algorithm above. The new algorithm is given below and as we can see by solving this recurrence has a worst case time complexity that is linear. This is a pretty impressive algorithm and complexity and it took quite a bit of work to find it. Unfortunately as {cite:p}`cormen2022introduction` say it is not as practical as the earlier randomized version of this algorithm but is still of great theoretical interest. 

```{prf:algorithm} Selection - $O(n)$ Worst Case Time
:class: dropdown
:label: selection $O(n)$ Worst Case Time
**procedure** $\text{selection}(L, k)$:

**Inputs** Given a list $L[1,...,n]$ of length $n$ and an integer $1\leq k \leq n$

**Output** The $k$-th smallest element in the list

1. Break $L$ into $\frac{n}{5}$ groups, $G_1,G_2,...,G_{n/5}$ $\quad \triangleright$ Lines 1-4 are for selecting the pivot $w$
2. For $i=1$ to $\frac{n}{5}$:  
   1. Sort $G_i$ $\quad \triangleright$ Sort each $G_i$
   2. Set $m_i=\text{median}(G_i)$ $\quad \triangleright$ Set $m_i$ to median of sorted list $G_i$
3. $S=[m_1,m_2,...,m_{n/5}]$ $\quad \triangleright$ Create new list of median values
4. $w = \text{selection}(S, \frac{n}{10})$ $\quad \triangleright$ Recursively call algorithm to select a good pivot from this sample

5. Divide $L$ into the lists $L_l$, $L_w$ and $L_g$
6. if $k \leq |L_l|$:
   1. return $\text{selection}(L_l, k)$
7. if $k > |L_l| + |L_w|$:
   1. return $\text{selection}(L_g, k-|L_l| - |L_w|)$
9. else:
    1. return $w$

#### Proof of good pivot selection

But why does the median of this list of medians, $S$, yield a good pivot? First note that since each group $G_i$ is of size $5$ if we sort it and take its median then we know exactly $2$ elements will be greater than or equal to $m_i$ and exactly $2$ elements will be less than or equal to $m_i$ so this means each element of $S$ will possess the property we said we wanted earlier. 

## Quicksort

Some of the ideas behind the median algorithm we just covered can also be applied to create an asymptotically efficient and empirically performant divide and conquer sorting algorithm called Quicksort. This algorithm, along with the one we will cover in the next section, was named as one of the top 10 with the greatest influence on the development and practice of science and engineering in the 20th century by [IEEE](https://www.computer.org/csdl/magazine/cs/2000/01/c1022/13rRUxBJhBm). 

## Fast Fourier transform

The fast (discrete) Fourier transform (FFT) is an algorithm that has revolutionized signal processing. {cite:p}`dasgupta2008algorithms` explains how one can 

The continuous time Fourier transform, Fourier transform for short, of a function $x(t)$, denoted as $\hat{x}(f)$ is given by

$$
\hat{x}(f) = \int_{-\infty}^{\infty}x(t)e^{-i2\pi f t}dt
$$

and the discrete time Fourier transform of a function $x(t)$, denoted as $\hat{x}(f)$ is given by

$$
\hat{x}(f) = \sum_{n=-\infty}^{\infty}x[n]e^{-i2\pi f t}.
$$

The *discrete* Fourier transform transforms a discrete sequence of complex numbers $\{x_0,x_1,...,x_{N-1}\}$ into another discrete sequence of complex numbers 
$\{X_0, X_1,...,X_{N-1}\}$ such that

$$
X_k = \sum_{n=0}^{N-1}x_n e^{-i2\pi k n /N}.
$$

For the FFT we are interested in 

### Representations of polynomials 

We first started by looking at how to multiply integers efficiently using a divide and conquer approach followed by how to multiply matrices. For the FFT we will extend this to the multiplication of *polynomials*.

#### Coefficient representation

Recall that a general polynomial $A(x)$ of degree $n-1$ can be represented as

$$
A(x) = a_0 + a_1x + a_2x^2 + \cdots + a_{n-1}x^{n-1}
$$

where $a_i \in \{a_0,...,a_{n-1}\}$ are coefficients. 

Given another polynomial $B(x) = b_0 + b_1x + b_2x^2 + \cdots + b_{n-1}x^{n-1}$ we want an algorithm that can compute their product

$$
\begin{align*}
C(x) &= A(x)B(x) \\
&= c_0 + c_1x + c_2x^2 + \cdots + c_{2n-2}x^{2n-2}
\end{align*}
$$

Note that the resulting function will be a polynomial of degree at most $2n-2$ and the coefficients have the form

$$
c_i = a_0b_i + a_1b_{i-1} + \cdots + a_ib_0 = \sum_{k=0}^ia_kb_{i-k}
$$

Note that if $i>n-1$ then $b_i=0$ and $a_i=0$ since $A(x)$ and $B(x)$ only have coefficients up to $a_{n-1}$ and $b_{n-1}$ respectively. This operation is actually known as a convolution and is denoted as $a\ast b$. Computing $c_i$ takes $O(i)$ time, where $i\leq 2n-2$, and there are at most $2n-1$ coefficients $c_i$ so this formula leads to an $O((2n-2)(2n-1))=O(n^2)$ time algorithm for multiplying polynomials. 

We can notice something interesting here: each of these polynomials is uniquely determined by its coefficients $(a_0,a_1,...,a_{n-1})$ and $(b_0,b_1,...,b_{n-1})$ and the resulting product can be computed using only these coefficients. Thus we can represent a polynomial as a vector of coefficients and their product as the convolution between these lists of coefficients.



#### Value representation


In addition to this there is another way to represent a polynomial and that is as $A(x) := (A(x_1),...,A(x_n))$. That is as a vector of values the polynomial takes on at the points $x_1,x_2,...x_{n}$ where all $x_i$ are *distinct*. These $n$ points also uniquely determine a polynomial of degree $n-1$. This representation is more useful for multiplying polynomials and what the FFT does is switch between these representations to multiply polynomials more efficiently than our $O(n^2)$ time from using the coefficient representation. 

But why is it more efficient to use this representation? Well if we have two value vectors $A(x) := (A(x_1),...,A(x_{2n}))$ and $B(x) := (B(x_1),...,B(x_{2n}))$ for two polynomials their product polynomial is uniquely characterized by the vector that results from taking the pairwise product of elements of the two vectors so $C(x) := (A(x_1)B(x_1),...,A(x_{2n})B(x_{2n}))$. Each of these products takes $O(1)$ time to compute since we are just multiplying two scalars and there are $2n$ of these products to compute so the time complexity is $O(n)$! Why are there $2n$ elements in these vectors instead of $n$? Well recall that the product of two $n$ degree polynomials has degree of at most $2n$ so we need to have at least $2n$ points in our vectors for $A(x)$ and $B(x)$ to uniquely characterize their resulting product polynomial.

Given the coefficient representation of a polynomial and $2n$ points $X=\{x_1,...,x_{2n}\}$ we want to evaluate the polynomial at the naive approach to switch to the value representation would involve evaluating the polynomial at each of these points by computing 

$$
A(x_i) = a_0 + a_1x_i + \cdots + a_{n-1}x_i^{n-1}
$$

for each $x_i$. This is naive since computing $A(x_i)$ takes $O(n)$ scalar multiplication and computing all $2n$ values of $A(x_i)$ thus yields a time complexity of $O(n^2)$ so we may as well have just used the coefficients to find the product of the polynomial instead of going through the trouble of converting to the value representation.

The FFT, by picking a clever set of points $X=\{x_1,...,x_{2n}\}$, will allow us to switch from the coefficient representation to the value representation in $O(n\log{n})$ time, after which we can do the multiplication in $O(n)$ time and then switch back the coefficient representation in $O(n\log{n})$ time for an overall time complexity of $O(n\log{n})$.

### Choosing the set $X$

## Practice problems

### 2.2

### 2.4

### 2.5

### 2.6

### 2.7

### 2.12

### 2.14

### 2.15

### 2.17 $A[i]=i$

### 2.18

### 2.19

### 2.20

### 2.22

In [1]:
%load_ext watermark
%watermark -n -u -v -iv

Last updated: Sun Jul 14 2024

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.22.2

